<a href="https://colab.research.google.com/github/RealAntonVoronov/computational_humour/blob/master/from_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинный перевод с помощью openNMT для соревнования STAPLE от Duolingo

## Загрузка данных. 

Так как данных предоставленных орагнизаторами явно недостаточно для того чтобы обучить полноценную языковую модель, для каждого из 5 языков были загружены параллельные корпуса субтитров. (http://opus.nlpl.eu/OpenSubtitles-v2016.php) Данные можно также найти на сервере nlp1 в папке `voronov/data/OpenSubtitles`.

In [1]:
# this is for colab skip if you don't need to connect to drive)
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
import os
os.chdir('gdrive/My Drive/data/work/Panchenko/corpora/OpenSubtitles')
!mkdir en_hu
os.chdir('en_hu/')

mkdir: cannot create directory ‘en_hu’: File exists


In [6]:
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2016/moses/en-hu.txt.zip
!unzip  'download.php?f=OpenSubtitles%2Fv2016%2Fmoses%2Fen-hu.txt.zip'

Archive:  download.php?f=OpenSubtitles%2Fv2016%2Fmoses%2Fen-hu.txt.zip
  inflating: OpenSubtitles.en-hu.en  
  inflating: OpenSubtitles.en-hu.hu  
  inflating: OpenSubtitles.en-hu.ids  


Для правильного запуска openNMT понадобится разбить данные на трейн часть и на валидационную. Я решил использовать всего 1 миллион пар предложений для обучения и 5000 пар для валидации (это не в прямом смысле валидация, она нужна только для корректной работы скриптов openNMT, в их руководстве сказано, что 5000 будет достаточно)  

In [0]:
!head -n 1000000 'OpenSubtitles.en-hu.en' > 'train_subtitles_1m_en.txt'
!head -n 1000000 'OpenSubtitles.en-hu.hu' > 'train_subtitles_1m_hu.txt'
!tail -n 5000 'OpenSubtitles.en-hu.en' > 'dev_subtitles_en.txt'
!tail -n 5000 'OpenSubtitles.en-hu.hu' > 'dev_subtitles_hu.txt'

## Данные Duolingo

Для составления предсказания для соревнования Duolingo я взял файл формата `*.aws_baseline.pred.txt`, который содержит по одному референсному переводу от Amazon для каждого предложения, предобработал его (убрал префиксы у строчек, соединил строчки "сообщение-перевод" в одну, разделённую табом, удалил пустые строчки) и разбил на два файла: с исходным языком и таргетным. Для того чтобы на стадии предсказания точно не возникло незнакомых для модели слов, я добавил полученные файлы в обучающую выборку.

In [5]:
os.chdir('../../../duolingo/data/en-hu/train')
#remove all markers | remove blank lines | combine every two lines in one
!sed 's/.*|//g' train.en_hu.aws_baseline.pred.txt | grep . | paste -d "\t"  - - > train_duolingo_en_hu.txt

train_duolingo_en_hu.txt	   train.en_hu.gold.txt
train_duolingo_en.txt		   train-sents.clean.bpe.en
train_duolingo_hu.txt		   train-sents.clean.bpe.hu
train.en_hu.aws_baseline.pred.txt


In [0]:
with open('train_duolingo_en_hu.txt') as f, open('train_duolingo_en.txt', 'w') as file_en, \
open('train_duolingo_hu.txt', 'w') as file_hu:
    for line in f.readlines():
        pair = line.strip().split('\t')
        file_en.writelines(pair[0]+'\n')
        file_hu.writelines(pair[1]+'\n')

In [6]:
!tail -5 ../../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_en.txt
!cat train_duolingo_en.txt >> ../../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_en.txt
!cat train_duolingo_hu.txt >> ../../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_hu.txt
!tail -5 ../../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_en.txt

Come on. - Sir.
Digger?
Gentle with him lads.
See if the back is clear.
Can I help?
that game would have changed my life.
i do not want him to open the window.
we have thousands of offices.
he goes to school.
and the patients are old.


## Обучение

In [1]:
!pip install OpenNMT-py

In [0]:
import os
os.chdir('../../../language_models/en_hu/openNMT')

In [11]:
!onmt_preprocess -train_src ../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_en.txt \
-train_tgt ../../../corpora/OpenSubtitles/en_hu/train_subtitles_1m_hu.txt \
-valid_src ../../../corpora/OpenSubtitles/en_hu/dev_subtitles_en.txt \
-valid_tgt ../../../corpora/OpenSubtitles/en_hu/dev_subtitles_hu.txt \
-tgt_vocab_size 50000 -src_vocab_size 50000 --src_seq_length 25 --tgt_seq_length 25 \
-save_data nmt_subs_en_hu

[2020-03-17 13:54:28,653 INFO] Extracting features...
[2020-03-17 13:54:28,660 INFO]  * number of source features: 0.
[2020-03-17 13:54:28,660 INFO]  * number of target features: 0.
[2020-03-17 13:54:28,660 INFO] Building `Fields` object...
[2020-03-17 13:54:28,660 INFO] Building & saving training data...
[2020-03-17 13:54:29,864 INFO] Building shard 0.
[2020-03-17 13:54:56,612 INFO]  * saving 0th train data shard to openNMT/nmt_subs_en_hu.train.0.pt.
[2020-03-17 13:55:15,907 INFO]  * tgt vocab size: 50004.
[2020-03-17 13:55:16,181 INFO]  * src vocab size: 50002.
[2020-03-17 13:55:17,316 INFO] Building & saving validation data...
[2020-03-17 13:55:18,179 INFO] Building shard 0.
[2020-03-17 13:55:18,307 INFO]  * saving 0th valid data shard to openNMT/nmt_subs_en_hu.valid.0.pt.


Это облегченная модель трансформера (чтобы обучение занимало адекватное время (~12 часов). Я уменьшил `rnn_size`, `word_vec_size`, `heads`, `training_steps` в 2 раза. Остальное оставил таким же как указано в рекомендациях openNMT, которые якобы повторяют исходный сетап Google.

In [0]:
!onmt_train -data nmt_subs_en_hu -save_model transformer \
        -layers 6 -rnn_size 256 -word_vec_size 256 -transformer_ff 2048 -heads 4  \
        -encoder_type transformer -decoder_type transformer -position_encoding \
        -train_steps 100000  -max_generator_batches 2 -dropout 0.1 \
        -batch_size 4096 -batch_type tokens -normalization tokens  -accum_count 2 \
        -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 \
        -max_grad_norm 0 -param_init 0  -param_init_glorot \
        -label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 10000 \
        -world_size 1 -gpu_ranks 0
# i omitted output with training statistics for better representation

Теперь у нас есть модель-переводчик, исходный текст для перевода и их референсный перевод от Amazon. Для примера выведем 10 лучших переводов одного предложения (автоматическое определение правильного числа переводов для каждого предложения -- потом)

In [0]:
os.chdir('../../../../language_models/en_hu/openNMT')
!onmt_translate -model transformer_step_50000.pt -src ../../../duolingo/data/en-hu/train/train_duolingo_en.txt \
-output train_duolingo_pred10_hu.txt -n_best 10 -replace_unk

In [12]:
!head -n 11 train_duolingo_pred10_hu.txt

Vannak animals?
Vannak Szereti az animals?
Szereti az animals?
Vannak animals? ?
Vannak egész animals?
Vannak Szereti az Szereti az animals?
Vannak Szereti az animals? .
Vannak Szereti az animals? - lgen.
Vannak Szereti az animals? - Igen.
Vannak Szereti az animals? - Vannak animals?
skin van.


## TODO: Evaluation



In [0]:
"""from nltk.translate.bleu_score import corpus_bleu
import re

references = []
predictions = []
with open('../../duolingo/data/en-pt/train/train_duolingo_pt.txt') as gold:
    for sentence in gold.readlines():
        s = ' '.join(re.split(r'([^0-9a-zÀ-ÿ\s])',sentence.lower()))
        s = re.sub(r' +', ' ', s)
        references.append(s.split())
with open('openNMT/train_duolingo_pred_pt.txt') as preds:
    for sentence in preds.readlines():
        s = ' '.join(re.split(r'([^0-9a-zÀ-ÿ\s])',sentence.lower()))
        s = re.sub(r' +', ' ', s)
        predictions.append(s.split())
print(predictions[1])
print(references[1])"""

['a', 'livraria', 'não', 'está', 'nesta', 'rua', '.']
['a', 'livraria', 'não', 'é', 'nesta', 'rua', '.']


In [0]:
# corpus_bleu(references, predictions)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6902782199046656